In [4]:
from __future__ import division

from models import *
from utils.utils import *
from utils.datasets import *
from utils.parse_config import *

import os
import sys
import time
import datetime
import argparse
import tqdm
import easydict

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
import torch.optim as optim


def evaluate(model, path, iou_thres, conf_thres, nms_thres, img_size, batch_size):
    model.eval()

    # Get dataloader
    dataset = ListDataset(path, img_size=img_size, augment=False, multiscale=False)
    dataloader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=False, num_workers=1, collate_fn=dataset.collate_fn
    )

    Tensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor

    labels = []
    sample_metrics = []  # List of tuples (TP, confs, pred)
    for batch_i, (_, imgs, targets) in enumerate(tqdm.tqdm(dataloader, desc="Detecting objects")):

        # Extract labels
        labels += targets[:, 1].tolist()
        # Rescale target
        targets[:, 2:] = xywh2xyxy(targets[:, 2:])
        targets[:, 2:] *= img_size

        imgs = Variable(imgs.type(Tensor), requires_grad=False)

        with torch.no_grad():
            outputs = model(imgs)
            outputs = non_max_suppression(outputs, conf_thres=conf_thres, nms_thres=nms_thres)

        sample_metrics += get_batch_statistics(outputs, targets, iou_threshold=iou_thres)

    # Concatenate sample statistics
    true_positives, pred_scores, pred_labels = [np.concatenate(x, 0) for x in list(zip(*sample_metrics))]
    precision, recall, AP, f1, ap_class = ap_per_class(true_positives, pred_scores, pred_labels, labels)

    return precision, recall, AP, f1, ap_class


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    args = parser.parse_args(args=[])
#     parser.add_argument("--batch_size", type=int, default=8, help="size of each image batch")
#     parser.add_argument("--model_def", type=str, default="config/yolov3.cfg", help="path to model definition file")
#     parser.add_argument("--data_config", type=str, default="config/coco.data", help="path to data config file")
#     parser.add_argument("--weights_path", type=str, default="weights/yolov3.weights", help="path to weights file")
#     parser.add_argument("--class_path", type=str, default="data/coco.names", help="path to class label file")
#     parser.add_argument("--iou_thres", type=float, default=0.5, help="iou threshold required to qualify as detected")
#     parser.add_argument("--conf_thres", type=float, default=0.001, help="object confidence threshold")
#     parser.add_argument("--nms_thres", type=float, default=0.5, help="iou thresshold for non-maximum suppression")
#     parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
#     parser.add_argument("--img_size", type=int, default=416, help="size of each image dimension")
#     opt = parser.parse_args()
    args = easydict.EasyDict({
        "batch_size": 6,
        "model_def":r'D:/Ivan/Test_data/IvanMadeDataSet/Yolo_front/config/yolov3_832.cfg',
        "data_config":r'D:/Ivan/Test_data/IvanMadeDataSet/Yolo_front_test/config/coco.data',
        "weights_path":r'D:/Ivan/YoloCheckpoints/OID_front_1_832/70.weights',
        "class_path": r'D:/Ivan/Test_data/IvanMadeDataSet/Yolo_front/config/coco.names',
        "iou_thres": 0.5,
        "conf_thres":0.8,
        "nms_thres": 0.5,
        "n_cpu": 2,
        "img_size": 832
    })
    opt = args
    print(opt)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
######## make a new data set
    data_config = parse_data_config(opt.data_config)
    valid_path = data_config["test"]
#     class_names = load_classes(data_config["names"])
    class_names= load_classes(opt.class_path)

    # Initiate model
    model = Darknet(opt.model_def).to(device)
    if opt.weights_path.endswith(".weights"):
        # Load darknet weights
        model.load_darknet_weights(opt.weights_path)
    else:
        # Load checkpoint weights
        model.load_state_dict(torch.load(opt.weights_path))

    print("Compute mAP...")

    precision, recall, AP, f1, ap_class = evaluate(
        model,
        path=valid_path,
        iou_thres=opt.iou_thres,
        conf_thres=opt.conf_thres,
        nms_thres=opt.nms_thres,
        img_size=opt.img_size,
        batch_size=opt.batch_size,
    )

    print("Average Precisions:")
    for i, c in enumerate(ap_class):
        print(f"+ Class '{c}' ({class_names[c]}) - AP: {AP[i]}")

    print(f"mAP: {AP.mean()}")
    print(f"precision: {precision.mean()}")
    print(f"recall: {recall.mean()} ")


{'batch_size': 6, 'model_def': 'D:/Ivan/Test_data/IvanMadeDataSet/Yolo_front/config/yolov3_832.cfg', 'data_config': 'D:/Ivan/Test_data/IvanMadeDataSet/Yolo_front_test/config/coco.data', 'weights_path': 'D:/Ivan/YoloCheckpoints/OID_front_1_832/70.weights', 'class_path': 'D:/Ivan/Test_data/IvanMadeDataSet/Yolo_front/config/coco.names', 'iou_thres': 0.5, 'conf_thres': 0.8, 'nms_thres': 0.5, 'n_cpu': 2, 'img_size': 832}
Compute mAP...


Computing AP: 100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.35it/s]


Average Precisions:
+ Class '0' (cars) - AP: 0.023193602080831703
mAP: 0.023193602080831703
precision: 0.008736296422197672
recall: 0.5473908413205538 


In [5]:
print('D:/Ivan/YoloCheckpoints/OID_front_1_832/'+str(70)+''.weights')

SyntaxError: EOL while scanning string literal (<ipython-input-5-8230e8ce4454>, line 1)